In [2]:
### Please Note, this code takes about 4 minutes to run ###
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from windpowerlib.wind_turbine import load_turbine_data_from_oedb 
%matplotlib inline
import os
from windpowerlib.modelchain import ModelChain 
from windpowerlib.wind_turbine import WindTurbine 
from windpowerlib import wind_turbine as wt

def get_weather_data(filename='weather.csv', **kwargs): 
    # change weather.csv into panda data frame
    if 'datapath' not in kwargs:
        kwargs['datapath'] = os.path.join(os.path.split(
            os.path.dirname(__file__))[0], 'example') 
    file = os.path.join(kwargs['datapath'], filename)
    # read csv file
    weather_df = pd.read_csv(file, index_col=0, header=[0, 1],date_parser=lambda idx: pd.to_datetime(idx, utc=True))
    # change type of index to datetime and set time zone
    weather_df.index = pd.to_datetime(weather_df.index).tz_convert( 'Europe/Berlin')
    # change type of height from str to int by resetting columns
    l0 = [_[0] for _ in weather_df.columns]
    l1 = [int(_[1]) for _ in weather_df.columns] 
    weather_df.columns = [l0, l1]
    return weather_df

weather = get_weather_data(filename='weather.csv', datapath='') 

WT_new = pd.read_csv('WT_valid.csv', low_memory=False) 

# WT_valid contains wind turbine models with the following information:
# manufacturer, turbine name, power curver, power coefficient curve

# The origincal dataset could be retried using: WT = wt.load_turbine_data_from_oedb(). 
# However, there are many models with missing key parameters, I downloaded the datatable as csv 
# and used Excel to tidy it up.

n = 1
x1 = [] # initialise turbine_type, hub_height as class storage
x2 = [] # initialise conveted model chain parameters storage
for n in range(len(WT_new)):
    try:
        L1 = WT_new['hub_height'].values[n].split(';')
        i = 0
        for i in range(len(L1)):
            x = {
                'turbine_type': str(WT_new['name'].values[n]),
                'hub_height':int(L1[i])
            }
            x1.append(x)
    except:
        pass
    
for j in range(len(x1)):
    y = WindTurbine(**x1[j])
    x2.append(y)
    
x3 = [] # initialise power output storage
modelchain_data = {
    'wind_speed_model': 'hellman',      # 'logarithmic' (default),
                                            # 'hellman' or
                                            # 'interpolation_extrapolation'
    'density_model': 'ideal_gas',           # 'barometric' (default), 'ideal_gas'
                                            #  or 'interpolation_extrapolation'
    'temperature_model': 'linear_gradient', # 'linear_gradient' (def.) or
                                            # 'interpolation_extrapolation'
    'power_output_model': 'power_curve',    # 'power_curve' (default) or
                                            # 'power_coefficient_curve'
    'density_correction': True,             # False (default) or True
    'obstacle_height': 0,                   # default: 0
    'hellman_exp': None}                    # None (default) or None
count = 0
for k in range(len(x2)):
    try: 
        # using "try...pass" ignores errors caused by incomplete power curve 
        #for few turbine models and skip them
        out = ModelChain(x2[k], **modelchain_data).run_model(weather)
        x3.append(out.power_output)
        count = count + 1
    except:
        pass 

PW = pd.DataFrame(x3) # initialise final output datatable
i = 0
idx1 = [] # store turbine model names (repetitions for different heights)
idx2 = [] # store turbine model heights
for i in range(len(x2)):
    x = x2[i].power_curve 
    if x is not None:
        idx1.append(x2[i].turbine_type)
        idx2.append(x2[i].hub_height)
PW.insert(0, "Model Name", idx1, True)
PW.insert(1,"Model Height",idx2, True)
PW.to_csv('WT_Pout.csv') # store the final hourly Power Output for all models based on wind speed profile.    

In [3]:
### This part randomly gives retail prices to the previously chosen wind turbine models.     ###
### Each model's price equals to its height, power and rotor diameter multiplied by a random ###
### coefficient to simulate difference in prices for similar models. In reality, this data   ###
### will be provided by the manufacturers should the user wish to choose from a selection    ###
### of models and manufacturers.                                                             ###

import random
price = pd.read_csv('model_height.csv', low_memory=False)
price = price.drop(["Unnamed: 0"],axis=1)
i = 0
cost = []
for i in range(len(price)):
    a = int(price['Model Height'].values[i])*round(random.uniform(1,1.2), 2)
    b = int(price['nominal_power'].values[i])*round(random.uniform(1,1.2), 2)
    c = int(price['rotor_diameter'].values[i])*round(random.uniform(1,1.2), 2)
    cost.append(200*(a+b+c))
price.insert(loc=5, column='cost', value=cost)
price.to_csv('price.csv') # store the price, rotor diameter, area and other information for Matlab.   